# Cours ENPC - Pratique du calcul scientifique

## Systèmes linéaires

In [ ]:
# Bibliothèques et configuration par défaut utiles pour le TD
# Cela n'empêche nullement d'installer et de faire appel à d'autres bibliothèques si nécessaire

using LinearAlgebra, Polynomials, Plots, LaTeXStrings

Plots.default(fontfamily="Computer Modern",
              titlefontsize=20,
              xlabelfontsize=20,
              ylabelfontsize=20,
              legendfontsize=12,
              xtickfontsize=12,
              ytickfontsize=12,
              framestyle=:box,
              linewidth=2,
              label=nothing,
              grid=true)

### Exercice sur la factorisation de Cholesky

L'objectif de cet exercice est de proposer un algorithme permettant de réaliser la factorisation de Cholesky d'une matrice réelle définie positive $\mathsf{A}\in\mathbb{R}^{n×n}$ i.e. d'identifier la matrice réelle triangulaire inférieure $\mathsf{C}$ telle que :
<a id="ACC"></a>
$$
\tag{1}
\mathsf{A}=\mathsf{C}\mathsf{C}^T
$$

1. Construire une fonction qui prend comme argument une matrice `A` et qui renvoie la matrice `C`.
   Pour calculer $\mathsf{C}$, s'appuyer sur une identification successive des colonnes de `C` en commençant par l'élément diagonal puis le reste de la colonne,
   en comparant les deux membres de <a href="#ACC">(1)</a>.

1. Construire une fonction permettant de générer une matrice définie positive aléatoire dans $\mathbb{R}^{n×n}$. On pourra par exemple générer une matrice aléatoire `B` avec `rand(n,n)` puis calculer `B'B+I` où l'identité `I` est l'objet `UniformScaling` de la bibliothèque `LinearAlgebra`. (*L'ajout de l'identité permet de s'affranchir des éventuelles valeurs propres trop proches de `0`.*)

1. Tester les algorithmes précédents par un code du type
    ```julia
    n = 1000
    A = generate_defpos_matrix(n)
    C = cholesky(A)
    norm(C*C' - A, Inf)
    ```

1. On suppose maintenant que la matrice réelle définie positive `A` est à largeur de bande `b` supposée beaucoup plus petite que `n`. Réécrire la fonction de décomposition de Cholesky en exploitant la largeur de bande.

1. Construire une fonction permettant de générer une matrice définie positive aléatoire à largeur de bande donnée `b`. On pourra commencer par générer une matrice aléatoire triangulaire inférieure `B` à largeur de bande `b` puis renvoyer `B'B+I`.

1. Tester les algorithmes des deux questions précédentes. On prendra par exemple `n=1000` et `b=4`.

1. *Optionnel :* réaliser une étude de la complexité des algorithmes non optimisé et optimisé pour matrices à largeur de bande donnée en traçant en échelle double log les temps de calcul pour différentes valeurs de `n`.

   *Suggestions :*
    - Prendre pour `n` des puissances de `2` de `2⁷` à `2¹⁰`

    - Utiliser la macro `@elapsed` pour enregistrer le temps de calcul d'un appel en faisant une moyenne sur un certain nombre de réalisations (par exemple `10`)

    - Exploiter la fonction `fit` de [`Polynomials.jl`](https://github.com/JuliaMath/Polynomials.jl) pour trouver la puissance de la complexité en la supposant de la forme $αn^\beta$.

In [ ]:
function cholesky(A)
    # Your code comes here
    return C
end

In [ ]:
function generate_defpos_matrix(n)
    B = rand(n,n)
    return B'B+I
end

In [ ]:
n = 1000
A = generate_defpos_matrix(n)
C = cholesky(A)
norm(C*C' - A, Inf)

In [ ]:
function cholesky_banded(A, b)
    # Your code comes here
end

In [ ]:
function generate_banded_matrix(n, b)
    C = [j≤i≤j+b ? rand() : zero(Float64) for i in 1:n, j in 1:n]
    return C*C'+I
end

n = 10 ; b = 4
A = generate_banded_matrix(n,b)
C = cholesky_banded(A,b)
norm(C*C' - A, Inf)

In [ ]:
nb_samples = 10
mean(x) = sum(x)/length(x)
plot(xaxis=:log10, yaxis=:log10, xlabel="n", ylabel="CPU time", legend=:topleft)
tf = Float64[] ; tb = Float64[]
tn = 2 .^(7:10)
for n in tn
    A = generate_banded_matrix(n, b)
    println(n)
    push!(tf, mean(@elapsed cholesky(A) for _ in 1:nb_samples))
    push!(tb, mean(@elapsed cholesky_banded(A,b) for _ in 1:nb_samples))
end
Pf = fit(log10.(tn),log10.(tf),1) ; af = round(coeffs(Pf)[2]; digits=2)
plot!(tn, tf, marker=:o, label="Algo matrice pleine "*latexstring("n^{$(af)}"))
ntn = 2 .^(11:12)
for n in ntn
    A = generate_banded_matrix(n, b)
    println(n)
    push!(tb, mean(@elapsed cholesky_banded(A,b) for _ in 1:nb_samples))
end
append!(tn,ntn)
Pb = fit(log10.(tn),log10.(tb),1) ; ab = round(coeffs(Pb)[2]; digits=2)
plot!(tn, tb, marker=:diamond, label="Algo matrice bande "*latexstring("n^{$(ab)}"))

### Exercice: itération de Richardson

Considérer le système linéaire suivant:
$$
    \mathsf A \mathbf x :=
    \begin{pmatrix}
        3 & 1 \\ 1 & 3
    \end{pmatrix}
    \begin{pmatrix}
        x_1 \\
        x_2
    \end{pmatrix}
    =
    \begin{pmatrix}
        9 \\
        11
    \end{pmatrix} =: \mathbf b.
$$

 1. Illustrer à l'aide de la fonction `Plots.contourf` les lignes de niveau de la fonction
 $$
     f(\mathbf x) = \frac{1}{2} \mathbf x^T \mathsf A \mathbf x - \mathbf b^T \mathbf x.
 $$

 1. Implémenter l'itération de Richardson avec $\omega = 0.1$ pour résoudre le système,
    et illustrer les itérations au dessus des lignes de niveau de la fonction $f$.
    Comme critère d'arrêt, utiliser
    $$
    \lVert \mathsf A \mathbf x - \mathbf b \lVert \leq \varepsilon \lVert \mathbf b \lVert,
    $$
    evec $\varepsilon$ suffisamment petit.

 1. Faire un plot de l'erreur $e_k := \lVert \mathbf x^{(k)} - \mathbf x_* \rVert$ en fonction de $k$,
    en utilisant une échelle linéaire pour l'axe des abcisses et une échelle logarithmique pour l'axe des ordonnées,
    gràce à l'argument `yscale=:log` passé à la fonction `Plots.plot`.

 1. En utilisant `Polynomials.fit`, calculer une approximation du type
    $$
    \log(e_k) \approx a + bk \qquad \Leftrightarrow \qquad e_k \approx \exp(a) \times \exp(b)^k.
    $$
    Comparer la valeur de $\exp(b)$ au rayon spectral $\rho(\mathsf A - \omega \mathsf I)$ et expliquer.

 1. Calculer le paramètre $\omega$ optimal et refaire le plot de la décroissance de l'erreur dans ce cas.

In [ ]:
using LaTeXStrings
using LinearAlgebra
using Plots
using Polynomials

A = [3. 1.; 1. 3.]
b = [11.; 9.]
sol = A\b


### Exercice: itération de Gauss—Seidel

On s'intéresse dans cet exercice à la résolution de l'équation de Poisson en dimension 1 avec conditions de Dirichlet homogènes:
$$
- u''(x) = b(x), \qquad u(0) = u(1) = 0, \qquad b(x) := \exp(x).
$$
Une discrétisation de cette équation sur un maillage uniforme par la méthode des différences finies conduit au système linéaire
$$
\frac{1}{h^2}
\begin{pmatrix}
    2 & -1 \\
    -1 & 2  & -1 \\
       & -1 & 2      & -1 \\
       &    & \ddots & \ddots & \ddots & \\
       &    &        & -1    & 2      & -1 \\
       &    &        &     & -1      & 2 \\
\end{pmatrix}
\begin{pmatrix}
    u_1 \\
    u_2 \\
    u_3 \\
    \vdots \\
    u_{n-2} \\
    u_{n-1}
\end{pmatrix}
=
\begin{pmatrix}
    b(x_1) \\
    b(x_2) \\
    b(x_3) \\
    \vdots \\
    b(x_{n-2}) \\
    b(x_{n-1})
\end{pmatrix},
\qquad
h := \frac{1}{n},
\qquad
x_i := ih.
$$
où $h$ est l'espace entre les points de discrétisation et $(u_1, u_2, \cdots, u_{n-1})$ sont les valeurs recherchées de la fonction inconnue $u$ aux points intérieurs du domaine $[0, 1]$.

1. Calculer la solution du système pour $n = 50$ en utilisant la méthode `\` de Julia.

   *Indication*: pour construire la matrice du système linéaire,
   on pourra utiliser `LinearAlgebra.SymTridiagonal` ainsi que la fonction `fill`.

1. Implémenter la méthode de Gauss-Seidel (ou sa généralisation, la méthode de relaxation) afin de résoudre le système linéaire pour $n = 50$,
   en n'utilisant cette fois pas les fonctions `\` et `inv` de Julia ni aucune bibliothèque logicielle.
   On initialisera l'itération à $\mathbf x_0 = \mathbf 0$.

1. Vérifier sur un graphe que la solution approchée obtenue est proche de la solution exacte,
   qui est donnée par $$u(x) = \exp(x) - 1 - (\exp(1) - 1)x.$$